In [1]:
pip install psycopg2-binary


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 1.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.7 MB/s eta 0:00:00


In [17]:
import pandas as pd
import psycopg2 as pg2
import os

# PostgreSQL connection
conn = pg2.connect(
    dbname="stocks_db",
    user="postgres",  
    password="password", 
    host="localhost",
    port="5432"
)
cur = conn.cursor() 

In [18]:
# Directory with stock .txt files
data_dir = "C:/Users/HP/Downloads/Huge Stock Market Dataset/Stocks"
tickers = ["aapl.us", "msft.us", "googl.us", "amzn.us", "tsla.us"]

# Load each ticker into the table
for ticker in tickers:
    df = pd.read_csv(os.path.join(data_dir, f"{ticker}.txt"))
    df["Ticker"] = ticker.split(".")[0].upper()  # Add ticker column
    for index, row in df.iterrows():
        cur.execute("""
            INSERT INTO stock_prices (Date, Open, High, Low, Close, Volume, Adjusted_Close, Ticker)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Volume"], row["Open"], row["Ticker"]))
    conn.commit()

In [20]:
cur.close()
conn.close()

In [11]:
pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ---------- ----------------------------- 0.8/3.0 MB 1.5 MB/s eta 0:00:02
     -------------------- ------------------- 1.6/3.0 MB 2.1 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 2.3 MB/s eta 0:00:01
     -------------------------------------- - 2.9/3.0 MB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 2.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl

In [13]:
conn = psycopg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
cur.execute("SELECT column_name, constraint_name FROM information_schema.key_column_usage WHERE table_name = 'stock_prices';")
print(cur.fetchall())
cur.close()
conn.close()

[]


In [15]:
conn = psycopg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS stock_prices;")
cur.execute("""
    CREATE TABLE stock_prices (
        Date DATE,
        Open NUMERIC,
        High NUMERIC,
        Low NUMERIC,
        Close NUMERIC,
        Volume BIGINT,
        Adjusted_Close NUMERIC,
        Ticker VARCHAR(10),
        PRIMARY KEY (Date, Ticker)
    );
""")
conn.commit()
cur.close()
conn.close()

In [27]:
import yfinance as yf
import psycopg2

conn = psycopg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
for ticker in tickers:
    df = yf.download(ticker, start="2017-11-11", end="2025-03-13", auto_adjust=False)
    df["Ticker"] = ticker
    for index, row in df.iterrows():
        cur.execute("""
            INSERT INTO stock_prices (Date, Open, High, Low, Close, Volume, Adjusted_Close, Ticker)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (Date, Ticker) DO UPDATE SET
                Open = EXCLUDED.Open,
                High = EXCLUDED.High,
                Low = EXCLUDED.Low,
                Close = EXCLUDED.Close,
                Volume = EXCLUDED.Volume,
                Adjusted_Close = EXCLUDED.Adjusted_Close
        """, (
            index.date(),
            row["Open"].item(),
            row["High"].item(),
            row["Low"].item(),
            row["Close"].item(),
            int(row["Volume"].item()),
            row["Adj Close"].item(),
            ticker
        ))
    conn.commit()

cur.close()
conn.close()

[*********************100%***********************]  1 of 1 completed
Failed to get ticker 'MSFT' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Handle timeouts by increasing the timeout duration and adding retries.

Verify MSFT data separately to rule out persistent issues.

In [29]:
import yfinance as yf
import psycopg2
import time

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password", 
    host="localhost",
    port="5432"
)
cur = conn.cursor()

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
for ticker in tickers:
    retries = 3
    for attempt in range(retries):
        try:
            # Increase timeout and disable auto_adjust
            df = yf.download(ticker, start="2017-11-11", end="2025-03-13", auto_adjust=False, timeout=30)
            if df.empty:
                print(f"No data returned for {ticker}")
                break
            df["Ticker"] = ticker
            for index, row in df.iterrows():
                cur.execute("""
                    INSERT INTO stock_prices (Date, Open, High, Low, Close, Volume, Adjusted_Close, Ticker)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (Date, Ticker) DO UPDATE SET
                        Open = EXCLUDED.Open,
                        High = EXCLUDED.High,
                        Low = EXCLUDED.Low,
                        Close = EXCLUDED.Close,
                        Volume = EXCLUDED.Volume,
                        Adjusted_Close = EXCLUDED.Adjusted_Close
                """, (
                    index.date(),
                    row["Open"].item(),
                    row["High"].item(),
                    row["Low"].item(),
                    row["Close"].item(),
                    int(row["Volume"].item()),
                    row["Adj Close"].item(),
                    ticker
                ))
            conn.commit()
            print(f"Loaded data for {ticker}")
            break  # Exit retry loop on success
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {ticker}: {e}")
            if attempt < retries - 1:
                time.sleep(5)  # Wait before retrying
            else:
                print(f"Failed to load {ticker} after {retries} attempts")
                continue

cur.close()
conn.close()

[*********************100%***********************]  1 of 1 completed


Loaded data for AAPL


[*********************100%***********************]  1 of 1 completed


Loaded data for MSFT


[*********************100%***********************]  1 of 1 completed


Loaded data for GOOGL


[*********************100%***********************]  1 of 1 completed


Loaded data for AMZN


[*********************100%***********************]  1 of 1 completed


Loaded data for TSLA


In [31]:
#This confirms which tickers loaded and if MSFT data made it in.
conn = psycopg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
cur.execute("SELECT DISTINCT Ticker FROM stock_prices;")
print("Tickers in database:", cur.fetchall())
cur.execute("SELECT MAX(Date) FROM stock_prices WHERE Ticker = 'MSFT';")
print("Latest MSFT date:", cur.fetchall())
cur.close()
conn.close()

Tickers in database: [('AMZN',), ('AAPL',), ('GOOGL',), ('MSFT',), ('TSLA',)]
Latest MSFT date: [(datetime.date(2025, 3, 12),)]


Extracting 2015 - 2025 data

In [37]:
import psycopg2 as pg2
import pandas as pd

conn = pg2.connect(
    dbname="stocks_db",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
query = """
    SELECT Ticker, Date, Close, Volume
    FROM stock_prices
    WHERE Ticker IN ('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA')
    AND Date BETWEEN '2015-01-01' AND '2025-03-12'
    ORDER BY Ticker, Date;
"""
df = pd.read_sql(query, conn)
df.to_csv("portfolio_data.csv", index=False)
conn.close()

C:\Users\HP\AppData\Local\Temp\ipykernel_4288\1211015042.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [41]:
df = pd.read_csv('portfolio_data.csv')

In [43]:
df.head()

,ticker,date,close,volume
0,AAPL,2015-01-02,103.50,56196458
1,AAPL,2015-01-05,100.58,67880159
2,AAPL,2015-01-06,100.59,69460979
3,AAPL,2015-01-07,102.01,42324520
4,AAPL,2015-01-08,105.93,62679241


In [45]:
df.info

<bound method DataFrame.info of       ticker        date       close     volume
0       AAPL  2015-01-02  103.500000   56196458
1       AAPL  2015-01-05  100.580000   67880159
2       AAPL  2015-01-06  100.590000   69460979
3       AAPL  2015-01-07  102.010000   42324520
4       AAPL  2015-01-08  105.930000   62679241
...      ...         ...         ...        ...
12810   TSLA  2025-03-06  263.450012   98451600
12811   TSLA  2025-03-07  262.670013  102369600
12812   TSLA  2025-03-10  222.149994  189076900
12813   TSLA  2025-03-11  230.580002  174896400
12814   TSLA  2025-03-12  248.089996  142215700

[12815 rows x 4 columns]>

In [47]:
df.describe()

,close,volume
count,12815.000000,1.281500e+04
mean,215.680296,5.653505e+07
std,211.510748,5.795422e+07
min,11.931333,5.211410e+05
25%,88.211250,2.095500e+07
50%,148.500000,3.850540e+07
75%,242.839996,7.751545e+07
max,1132.880000,9.140820e+08
